<a href="https://colab.research.google.com/github/liumOazed/Trading-Strategy/blob/main/Simple_Momentum_%2B_ML_strategies_to_generate_Trading_Signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount drives
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
# Set your working directory to the notebook's folder
os.chdir("/content/drive/MyDrive/_Learning/Finance/Trading/Machine Learning for Trading - Specialization/2. Using ML in Trading & Finance/Module 4: Build a Momentum-based Trading System/")

# 1. Traditional Momentum Strategy

In [1]:
"""
================================================================================
MOMENTUM TRADING BACKTESTER
A clean, library-independent backtesting system for momentum-based strategies
================================================================================
"""

# ================================================================================
# SECTION 1: LIBRARY IMPORTS
# ================================================================================
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

In [2]:
# ================================================================================
# SECTION 2: DATA ACQUISITION
# ================================================================================

def download_stock_data(symbol='AAPL', start_date='2014-06-01', end_date='2017-09-01'):
    """
    Download historical stock data from Yahoo Finance

    Args:
        symbol: Stock ticker symbol
        start_date: Start date in YYYY-MM-DD format
        end_date: End date in YYYY-MM-DD format

    Returns:
        DataFrame with OHLCV data
    """
    print(f"📥 Downloading {symbol} data from {start_date} to {end_date}...")

    # Download data
    df = yf.download(symbol, start=start_date, end=end_date, progress=False, auto_adjust=False)

    # Clean multi-level columns if present
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)

    # Remove timezone info if present
    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)

    # Standardize column names (lowercase, underscore-separated)
    df.columns = [c.replace(' ', '_').lower() for c in df.columns]

    # Create adjclose if it doesn't exist
    if 'adj_close' in df.columns:
        df['adjclose'] = df['adj_close']
    elif 'adjclose' not in df.columns and 'close' in df.columns:
        df['adjclose'] = df['close']

    print(f"✓ Downloaded {len(df)} trading days")
    return df

In [3]:
# ================================================================================
# SECTION 3: FEATURE ENGINEERING
# ================================================================================

def calculate_momentum_indicator(prices, period):
    """
    Calculate momentum: difference between current price and price N periods ago

    Args:
        prices: Series of prices
        period: Lookback period in days

    Returns:
        Series of momentum values
    """
    return prices - prices.shift(period)


def calculate_exponential_moving_average(prices, period):
    """
    Calculate Exponential Moving Average (EMA)

    Args:
        prices: Series of prices
        period: EMA period

    Returns:
        Series of EMA values
    """
    return prices.ewm(span=period, adjust=False).mean()


def engineer_features(df):
    """
    Calculate all technical indicators and features

    Features created:
        - mom_10: 10-day momentum
        - mom_30: 30-day momentum
        - ma_90: 90-day exponential moving average

    Args:
        df: DataFrame with price data

    Returns:
        DataFrame with added feature columns
    """
    print("\n🔧 Engineering technical features...")

    # Use adjusted close price for all calculations
    price = df['adjclose']

    # Momentum indicators (short and medium term)
    df['mom_10'] = calculate_momentum_indicator(price, period=10)
    df['mom_30'] = calculate_momentum_indicator(price, period=30)

    # Moving average (long term trend)
    df['ma_90'] = calculate_exponential_moving_average(price, period=90)

    # Remove rows with NaN values (warmup period for indicators)
    initial_rows = len(df)
    df = df.dropna()
    removed_rows = initial_rows - len(df)

    print(f"✓ Features calculated")
    print(f"  - Removed {removed_rows} warmup rows (indicator initialization)")
    print(f"  - {len(df)} valid rows remaining")

    return df

In [4]:
# ================================================================================
# SECTION 4: TRADING STRATEGY
# ================================================================================

def generate_trading_signals(df):
    """
    Generate buy/sell signals based on momentum strategy

    Strategy Logic:
        - BUY (signal=1): When 30-day momentum is positive
        - SELL (signal=0): When 30-day momentum is negative or zero

    Args:
        df: DataFrame with feature columns

    Returns:
        DataFrame with signal and position columns
    """
    print("\n📊 Generating trading signals...")

    # Initialize signal column
    df['signal'] = 0

    # Buy signal: positive 30-day momentum
    df.loc[df['mom_30'] > 0, 'signal'] = 1

    # Sell signal: non-positive 30-day momentum
    df.loc[df['mom_30'] <= 0, 'signal'] = 0

    # Calculate position changes (for detecting entry/exit points)
    df['position'] = df['signal'].diff()

    # Count signals
    buy_signals = (df['signal'] == 1).sum()
    sell_signals = (df['signal'] == 0).sum()

    print(f"✓ Signals generated:")
    print(f"  - {buy_signals} buy periods")
    print(f"  - {sell_signals} sell/hold periods")

    return df

In [5]:
# ================================================================================
# SECTION 4: TRADING STRATEGY
# ================================================================================

def generate_trading_signals(df):
    """
    Generate buy/sell signals based on momentum strategy

    Strategy Logic:
        - BUY (signal=1): When 30-day momentum is positive
        - SELL (signal=0): When 30-day momentum is negative or zero

    Args:
        df: DataFrame with feature columns

    Returns:
        DataFrame with signal and position columns
    """
    print("\n📊 Generating trading signals...")

    # Initialize signal column
    df['signal'] = 0

    # Buy signal: positive 30-day momentum
    df.loc[df['mom_30'] > 0, 'signal'] = 1

    # Sell signal: non-positive 30-day momentum
    df.loc[df['mom_30'] <= 0, 'signal'] = 0

    # Calculate position changes (for detecting entry/exit points)
    df['position'] = df['signal'].diff()

    # Count signals
    buy_signals = (df['signal'] == 1).sum()
    sell_signals = (df['signal'] == 0).sum()

    print(f"✓ Signals generated:")
    print(f"  - {buy_signals} buy periods")
    print(f"  - {sell_signals} sell/hold periods")

    return df

In [6]:
# ================================================================================
# SECTION 5: BACKTESTING ENGINE
# ================================================================================

class MomentumBacktester:
    """
    Backtesting engine for momentum trading strategies

    Tracks portfolio state, executes trades, and calculates performance metrics
    """

    def __init__(self, data, initial_capital=1000, capital_per_trade=100):
        """
        Initialize backtester

        Args:
            data: DataFrame with price data and signals
            initial_capital: Starting capital in dollars
            capital_per_trade: Maximum capital to use per trade
        """
        self.data = data.copy()
        self.initial_capital = initial_capital
        self.capital = initial_capital
        self.capital_per_trade = capital_per_trade

        # Portfolio state
        self.position = 0  # Number of shares currently held
        self.entry_price = 0  # Price at which current position was entered

        # Trade tracking
        self.trades = []  # List of all executed trades
        self.equity_curve = []  # Portfolio value over time

    def execute_backtest(self):
        """
        Run the backtest simulation

        Returns:
            Dictionary of performance metrics
        """
        print("\n🚀 Running backtest simulation...")

        trade_count = 0

        for idx, row in self.data.iterrows():
            current_price = row['adjclose']
            signal = row['signal']

            # Calculate current portfolio value
            position_value = self.position * current_price
            portfolio_value = self.capital + position_value

            # Record equity curve point
            self.equity_curve.append({
                'date': idx,
                'portfolio_value': portfolio_value,
                'capital': self.capital,
                'position': self.position,
                'price': current_price
            })

            # ============ ENTRY LOGIC ============
            if signal == 1 and self.position == 0:  # Buy signal and no position
                # Calculate shares to buy
                shares_to_buy = int(self.capital_per_trade / current_price)

                if shares_to_buy > 0 and self.capital >= shares_to_buy * current_price:
                    # Execute buy order
                    cost = shares_to_buy * current_price
                    self.position = shares_to_buy
                    self.capital -= cost
                    self.entry_price = current_price
                    trade_count += 1

                    self.trades.append({
                        'date': idx,
                        'type': 'BUY',
                        'price': current_price,
                        'shares': shares_to_buy,
                        'cost': cost,
                        'capital': self.capital
                    })

            # ============ EXIT LOGIC ============
            elif signal == 0 and self.position > 0:  # Sell signal and holding position
                # Execute sell order
                proceeds = self.position * current_price
                pnl = proceeds - (self.position * self.entry_price)
                self.capital += proceeds
                trade_count += 1

                self.trades.append({
                    'date': idx,
                    'type': 'SELL',
                    'price': current_price,
                    'shares': self.position,
                    'proceeds': proceeds,
                    'pnl': pnl,
                    'capital': self.capital
                })

                # Close position
                self.position = 0
                self.entry_price = 0

        # ============ CLOSE ANY REMAINING POSITION ============
        if self.position > 0:
            final_price = self.data.iloc[-1]['adjclose']
            proceeds = self.position * final_price
            pnl = proceeds - (self.position * self.entry_price)
            self.capital += proceeds
            trade_count += 1

            self.trades.append({
                'date': self.data.index[-1],
                'type': 'SELL (FINAL)',
                'price': final_price,
                'shares': self.position,
                'proceeds': proceeds,
                'pnl': pnl,
                'capital': self.capital
            })

            self.position = 0

        print(f"✓ Backtest complete: {trade_count} trades executed")

        # Calculate and return performance metrics
        return self.calculate_performance_metrics()

    def calculate_performance_metrics(self):
        """
        Calculate comprehensive performance metrics

        Returns:
            Dictionary containing all performance statistics
        """
        print("\n📈 Calculating performance metrics...")

        # Convert to DataFrames for easier analysis
        equity_df = pd.DataFrame(self.equity_curve)
        trades_df = pd.DataFrame(self.trades)

        # ============ BASIC RETURNS ============
        final_capital = self.capital
        total_return = final_capital - self.initial_capital
        pnl_percent = total_return / self.initial_capital

        # ============ TRADE STATISTICS ============
        if 'pnl' in trades_df.columns and len(trades_df) > 0:
            profitable_trades = trades_df[trades_df['pnl'] > 0]
            losing_trades = trades_df[trades_df['pnl'] < 0]

            total_profit = profitable_trades['pnl'].sum() if len(profitable_trades) > 0 else 0
            total_loss = abs(losing_trades['pnl'].sum()) if len(losing_trades) > 0 else 0
            count_profit = len(profitable_trades)
            count_loss = len(losing_trades)
        else:
            total_profit = 0
            total_loss = 0
            count_profit = 0
            count_loss = 0

        # ============ PORTFOLIO STATISTICS ============
        portfolio_values = equity_df['portfolio_value'].values

        # Daily returns
        returns = np.diff(portfolio_values) / portfolio_values[:-1]

        # Drawdown analysis
        running_max = np.maximum.accumulate(portfolio_values)
        drawdown_percent = (portfolio_values - running_max) / running_max
        max_drawdown_dollars = abs(drawdown_percent.min()) * self.initial_capital

        # ============ ANNUALIZED METRICS ============
        trading_days = len(self.data)
        years = trading_days / 252  # 252 trading days per year

        annual_return = pnl_percent / years if years > 0 else 0
        annual_vol = returns.std() * np.sqrt(252) if len(returns) > 0 else 0
        sharpe_ratio = annual_return / annual_vol if annual_vol > 0 else 0

        # ============ BENCHMARK COMPARISON ============
        # Buy and hold return
        benchmark_return = (self.data.iloc[-1]['adjclose'] - self.data.iloc[0]['adjclose']) / self.data.iloc[0]['adjclose']

        # Score: excess return over benchmark
        score = (pnl_percent - benchmark_return) / abs(benchmark_return) if benchmark_return != 0 else 0

        # ============ COMPILE RESULTS ============
        results = {
            'instrument_names': ['AAPL'],
            'instrument_stats': [{
                'pnl': {'AAPL': pnl_percent},
                'score': {'AAPL': score}
            }],
            'pnl': pnl_percent,
            'trading_days': trading_days,
            'annual_return': annual_return,
            'annual_vol': annual_vol,
            'sharpe_ratio': sharpe_ratio,
            'score': score,
            'capitalUsage': self.capital_per_trade,
            'total_profit': total_profit,
            'maxDrawdown': max_drawdown_dollars,
            'maxPortfolioValue': portfolio_values.max(),
            'total_loss': total_loss,
            'variance': returns.var() if len(returns) > 0 else 0,
            'capital': final_capital,
            'count_profit': count_profit,
            'portfolio_value': portfolio_values[-1],
            'count_loss': count_loss
        }

        print("✓ Metrics calculated")

        return results

In [7]:
# ================================================================================
# SECTION 6: RESULTS DISPLAY
# ================================================================================

def display_results(results, backtester):
    """
    Display backtest results in a formatted output

    Args:
        results: Dictionary of performance metrics
        backtester: MomentumBacktester instance
    """
    print("\n" + "="*70)
    print("BACKTEST RESULTS SUMMARY")
    print("="*70)

    # Key performance metrics
    print("\n📊 PERFORMANCE METRICS:")
    print(f"  Initial Capital:     ${backtester.initial_capital:,.2f}")
    print(f"  Final Capital:       ${results['capital']:,.2f}")
    print(f"  Total Return:        ${results['capital'] - backtester.initial_capital:,.2f}")
    print(f"  Return %:            {results['pnl']*100:.2f}%")
    print(f"  Annual Return:       {results['annual_return']*100:.2f}%")
    print(f"  Sharpe Ratio:        {results['sharpe_ratio']:.4f}")

    # Risk metrics
    print("\n⚠️  RISK METRICS:")
    print(f"  Max Drawdown:        ${results['maxDrawdown']:,.2f}")
    print(f"  Annual Volatility:   {results['annual_vol']*100:.2f}%")
    print(f"  Variance:            {results['variance']:.6f}")

    # Trading statistics
    print("\n📈 TRADING STATISTICS:")
    print(f"  Total Trades:        {results['count_profit'] + results['count_loss']}")
    print(f"  Winning Trades:      {results['count_profit']}")
    print(f"  Losing Trades:       {results['count_loss']}")

    if (results['count_profit'] + results['count_loss']) > 0:
        win_rate = results['count_profit'] / (results['count_profit'] + results['count_loss']) * 100
        print(f"  Win Rate:            {win_rate:.2f}%")

    print(f"  Total Profit:        ${results['total_profit']:,.2f}")
    print(f"  Total Loss:          ${results['total_loss']:,.2f}")

    # Additional info
    print("\n📅 PERIOD:")
    print(f"  Trading Days:        {results['trading_days']}")
    print(f"  Years:               {results['trading_days']/252:.2f}")

    print("\n" + "="*70)

In [8]:
#================================================================================
# SECTION 7: MAIN EXECUTION
# ================================================================================

def main():
    """
    Main execution function - orchestrates the entire backtesting pipeline
    """
    print("="*70)
    print("           MOMENTUM TRADING STRATEGY BACKTEST")
    print("="*70)

    # ========== STEP 1: DATA ACQUISITION ==========
    df = download_stock_data(
        symbol='AAPL',
        start_date='2014-06-01',
        end_date='2017-09-01'
    )

    # ========== STEP 2: FEATURE ENGINEERING ==========
    df = engineer_features(df)

    # ========== STEP 3: FILTER TO BACKTEST PERIOD ==========
    # (Allow warmup period for indicators, then start backtest)
    backtest_start = '2015-01-02'
    backtest_end = '2017-08-31'
    df = df.loc[backtest_start:backtest_end]

    print(f"\n📅 Backtest Period: {backtest_start} to {backtest_end}")
    print(f"   Total Days: {len(df)}")

    # ========== STEP 4: GENERATE SIGNALS ==========
    df = generate_trading_signals(df)

    # ========== STEP 5: RUN BACKTEST ==========
    backtester = MomentumBacktester(
        data=df,
        initial_capital=1000,
        capital_per_trade=100
    )

    results = backtester.execute_backtest()

    # ========== STEP 6: DISPLAY RESULTS ==========
    display_results(results, backtester)

    # ========== STEP 7: RETURN FULL RESULTS ==========
    print("\n📦 Full Results Dictionary:")
    print("="*70)
    print(results)

    return results

In [9]:
# ================================================================================
# SECTION 8: SCRIPT ENTRY POINT
# ================================================================================

if __name__ == "__main__":
    results = main()

           MOMENTUM TRADING STRATEGY BACKTEST
📥 Downloading AAPL data from 2014-06-01 to 2017-09-01...
✓ Downloaded 821 trading days

🔧 Engineering technical features...
✓ Features calculated
  - Removed 30 warmup rows (indicator initialization)
  - 791 valid rows remaining

📅 Backtest Period: 2015-01-02 to 2017-08-31
   Total Days: 672

📊 Generating trading signals...
✓ Signals generated:
  - 406 buy periods
  - 266 sell/hold periods

🚀 Running backtest simulation...
✓ Backtest complete: 54 trades executed

📈 Calculating performance metrics...
✓ Metrics calculated

BACKTEST RESULTS SUMMARY

📊 PERFORMANCE METRICS:
  Initial Capital:     $1,000.00
  Final Capital:       $1,025.66
  Total Return:        $25.66
  Return %:            2.57%
  Annual Return:       0.96%
  Sharpe Ratio:        0.6853

⚠️  RISK METRICS:
  Max Drawdown:        $22.68
  Annual Volatility:   1.40%
  Variance:            0.000001

📈 TRADING STATISTICS:
  Total Trades:        27
  Winning Trades:      11
  Losing 

# 2. ML Trading Strategy: Classification with Rolling Cross-Validation (RAW)


In [16]:
# -*- coding: utf-8 -*-
"""
Real ML Trading Strategy: Random Forest with Rolling Cross-Validation
With Overfitting Analysis (Train vs Test)
"""

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# ==============================================================================
# 1. DATA ACQUISITION
# ==============================================================================
def download_stock_data(symbol='AAPL', start_date='2010-01-01', end_date='2023-01-01'):
    print(f"📥 Downloading {symbol}...")
    df = yf.download(symbol, start=start_date, end=end_date, progress=False, auto_adjust=False)

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)

    df.columns = [c.replace(' ', '_').lower() for c in df.columns]

    # Ensure 'adjclose' exists
    if 'adj_close' in df.columns:
        df['adjclose'] = df['adj_close']
    elif 'close' in df.columns:
        df['adjclose'] = df['close']

    return df

# ==============================================================================
# 2. FEATURE ENGINEERING (X) & TARGET CREATION (Y)
# ==============================================================================
def prepare_ml_data(df):
    data = df.copy()

    # --- FEATURES (X) ---
    # 1. Momentum (Returns)
    data['ret_1d'] = data['adjclose'].pct_change()
    data['ret_5d'] = data['adjclose'].pct_change(5)

    # 2. Volatility (Risk)
    data['vol_20'] = data['ret_1d'].rolling(20).std()

    # 3. Simple Moving Average Ratio (Trend)
    data['ma_50'] = data['adjclose'].rolling(50).mean()
    data['dist_to_ma'] = (data['adjclose'] / data['ma_50']) - 1

    # 4. RSI
    delta = data['adjclose'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    data['rsi'] = 100 - (100 / (1 + rs))

    # --- TARGET (Y) ---
    data['target'] = (data['adjclose'].shift(-1) > data['adjclose']).astype(int)

    data = data.dropna()
    feature_cols = ['ret_1d', 'ret_5d', 'vol_20', 'dist_to_ma', 'rsi']

    return data, feature_cols

# ==============================================================================
# 3. BACKTESTING ENGINE (EXECUTION)
# ==============================================================================
class MLBacktester:
    def __init__(self, data, predictions):
        self.data = data.copy()
        self.data['pred_signal'] = predictions
        self.capital = 10000

    def run(self):
        # If signal is 1, we get the day's return. If 0, we get 0.
        # shift(1) because prediction made today applies to tomorrow's return
        self.data['strategy_ret'] = self.data['pred_signal'].shift(1) * self.data['ret_1d']
        self.data['cum_ret'] = (1 + self.data['strategy_ret']).cumprod()
        final_return = self.data['cum_ret'].iloc[-1] - 1
        return final_return

# ==============================================================================
# 4. ROLLING CROSS-VALIDATION WITH OVERFITTING CHECK
# ==============================================================================
def run_rolling_ml(df, feature_cols, train_months=24, test_months=6):

    print(f"\n🤖 TRAINING RANDOM FOREST (Rolling Window)")
    print(f"   Features: {feature_cols}")
    print("="*80)

    results = []

    start_index = 0
    train_size = train_months * 21
    test_size = test_months * 21

    # Initialize Model (max_depth=3 controls overfitting)
    model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)

    while start_index + train_size + test_size < len(df):

        # --- 1. SPLITTING ---
        train_data = df.iloc[start_index : start_index + train_size]
        test_data = df.iloc[start_index + train_size : start_index + train_size + test_size]

        if len(test_data) == 0: break

        X_train = train_data[feature_cols]
        y_train = train_data['target']

        X_test = test_data[feature_cols]
        y_test = test_data['target']

        # --- 2. TRAINING ---
        model.fit(X_train, y_train)

        # --- 3. OVERFITTING CHECK (TRAIN METRICS) ---
        # How well did it memorize the "Homework"?
        train_preds = model.predict(X_train)
        train_acc = accuracy_score(y_train, train_preds)

        # Calculate Train PnL (Theoretical)
        train_tester = MLBacktester(train_data, train_preds)
        train_pnl = train_tester.run()

        # --- 4. REALITY CHECK (TEST METRICS) ---
        # How well did it do on the "Final Exam"?
        test_preds = model.predict(X_test)
        test_acc = accuracy_score(y_test, test_preds)

        # Calculate Test PnL (Real)
        test_tester = MLBacktester(test_data, test_preds)
        test_pnl = test_tester.run()

        # Gap Analysis
        gap_acc = train_acc - test_acc

        # Regime Detection
        vol = test_data['ret_1d'].std() * np.sqrt(252)
        if vol > 0.30:
            regime_str = "High Volatility (Panic)"
        elif test_data['adjclose'].iloc[-1] > test_data['adjclose'].iloc[0]:
            regime_str = "Bull Market (Calm Up)"
        else:
            regime_str = "Bear Market (Calm Down)"

        # Store Results
        results.append({
            'Period_Start': test_data.index[0],
            'Period_End': test_data.index[-1],
            'Regime': regime_str,
            'Train_Acc': train_acc,
            'Test_Acc': test_acc,
            'Gap_Acc': gap_acc,
            'Train_PnL': train_pnl,
            'Test_PnL': test_pnl
        })

        # Slide forward
        start_index += test_size

    return pd.DataFrame(results)

# ==============================================================================
# 5. MAIN EXECUTION
# ==============================================================================
if __name__ == "__main__":
    # 1. Get Data
    raw_df = download_stock_data('AAPL', '2010-01-01', '2023-01-01')

    # 2. Prepare Features
    ml_df, features = prepare_ml_data(raw_df)

    # 3. Run ML
    results_df = run_rolling_ml(ml_df, features, train_months=36, test_months=6)

    # 4. Display Results (Formatted as requested)

    # Set Pandas to display percentages nicely
    pd.options.display.float_format = '{:,.2%}'.format

    # Select columns for the detailed view
    display_cols = ['Period_Start', 'Period_End', 'Regime', 'Train_Acc', 'Test_Acc', 'Gap_Acc', 'Train_PnL', 'Test_PnL']

    print("\n📉 PERIOD-BY-PERIOD RESULTS (Train vs Test)")
    print("="*120)
    print(results_df[display_cols].to_string(index=False))

    print("\n📊 ML RESULTS SUMMARY")
    print(f"Average PnL per period:   {results_df['Test_PnL'].mean():.2%}")
    print(f"Average Model Accuracy:   {results_df['Test_Acc'].mean():.2%}")
    print(f"Win Rate (Profitable Periods): {len(results_df[results_df['Test_PnL']>0]) / len(results_df):.0%}")

    print("\n📊 OVERFITTING SUMMARY")
    print(f"Average Train Acc: {results_df['Train_Acc'].mean():.2%}")
    print(f"Average Test Acc:  {results_df['Test_Acc'].mean():.2%}")
    print(f"Average Gap:       {results_df['Gap_Acc'].mean():.2%}")

    if results_df['Gap_Acc'].mean() > 0.10:
        print("⚠️ WARNING: High Overfitting (>10% Gap). Consider reducing max_depth or features.")
    else:
        print("✅ Model looks stable (Gap < 10%).")

📥 Downloading AAPL...

🤖 TRAINING RANDOM FOREST (Rolling Window)
   Features: ['ret_1d', 'ret_5d', 'vol_20', 'dist_to_ma', 'rsi']

📉 PERIOD-BY-PERIOD RESULTS (Train vs Test)
Period_Start Period_End                  Regime  Train_Acc  Test_Acc  Gap_Acc  Train_PnL  Test_PnL
  2013-03-18 2013-09-13   Bull Market (Calm Up)     63.89%    50.79%   13.10%    717.88%    15.14%
  2013-09-16 2014-03-17   Bull Market (Calm Up)     61.90%    46.03%   15.87%    543.30%     0.50%
  2014-03-18 2014-09-15   Bull Market (Calm Up)     66.80%    49.21%   17.59%    677.75%    20.82%
  2014-09-16 2015-03-17   Bull Market (Calm Up)     65.87%    53.97%   11.90%    402.83%    17.63%
  2015-03-18 2015-09-15 Bear Market (Calm Down)     65.74%    50.79%   14.95%    328.70%     1.66%
  2015-09-16 2016-03-16 Bear Market (Calm Down)     67.33%    46.83%   20.50%    332.64%   -12.88%
  2016-03-17 2016-09-14   Bull Market (Calm Up)     64.55%    47.62%   16.93%    231.43%    -5.90%
  2016-09-15 2017-03-16   Bull Mar

# 3. ML Trading Strategy: Classification with Rolling Cross-Validation (New Features + Normalized)

In [17]:
# -*- coding: utf-8 -*-
"""
Advanced ML Classification: Overfitting Analysis (Train vs Test)
Features: Hurst, MA Crossover, RSI, Correlation Filter
"""

from sklearn.preprocessing import StandardScaler

# ==============================================================================
# 1. HELPER: HURST EXPONENT (Numpy Optimized)
# ==============================================================================
def get_hurst_exponent(time_series):
    lags = range(2, 20)
    tau = []
    for lag in lags:
        if len(time_series) > lag:
            diff = np.subtract(time_series[lag:], time_series[:-lag])
            tau.append(np.sqrt(np.std(diff)))
        else:
            return 0.5
    try:
        m = np.polyfit(np.log(lags), np.log(tau), 1)
        return m[0] * 2.0
    except:
        return 0.5

# ==============================================================================
# 2. FEATURE ENGINEERING
# ==============================================================================
def prepare_features(df):
    print("🛠 Generating Technical Features...")
    data = df.copy()

    # Classic Momentum
    data['ma_15'] = data['adjclose'].rolling(window=15).mean()
    data['ma_150'] = data['adjclose'].rolling(window=150).mean()
    data['ma_spread'] = (data['ma_15'] - data['ma_150']) / data['ma_150']

    # MH Indicator (Hurst)
    log_ret = np.log(data['adjclose'] / data['adjclose'].shift(1))
    data['h16'] = log_ret.rolling(window=16).apply(get_hurst_exponent, raw=True)
    data['h32'] = log_ret.rolling(window=32).apply(get_hurst_exponent, raw=True)
    data['mh_diff'] = data['h32'] - data['h16']

    # RSI
    delta = data['adjclose'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    data['rsi'] = 100 - (100 / (1 + rs))

    # Standard
    data['ret_1d'] = data['adjclose'].pct_change()
    data['vol_20'] = data['ret_1d'].rolling(20).std()

    # Target
    data['target'] = (data['adjclose'].shift(-1) > data['adjclose']).astype(int)

    data = data.dropna()
    return data

# ==============================================================================
# 3. CORRELATION FILTER
# ==============================================================================
def remove_correlated_features(df, feature_cols, threshold=0.7):
    print("\n🔍 CHECKING FEATURE CORRELATION...")
    df_feat = df[feature_cols]
    corr_matrix = df_feat.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return [f for f in feature_cols if f not in to_drop]

# ==============================================================================
# 4. REGIME DETECTION
# ==============================================================================
def detect_regime(data_slice):
    vol = data_slice['ret_1d'].std() * np.sqrt(252)
    if vol > 0.30:
        return "High Volatility (Panic)"
    elif data_slice['adjclose'].iloc[-1] > data_slice['adjclose'].iloc[0]:
        return "Bull Market (Calm Up)"
    else:
        return "Bear Market (Calm Down)"

# ==============================================================================
# 5. ROLLING CLASSIFICATION (With Train vs Test Analysis)
# ==============================================================================
def run_rolling_classification(df, feature_cols):

    print(f"\n🚀 STARTING ROLLING BACKTEST (Checking Overfitting)")
    print("="*80)

    train_months = 24
    test_months = 6
    train_size = train_months * 21
    test_size = test_months * 21

    results = []
    model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)

    start_index = 0
    while start_index + train_size + test_size < len(df):

        # 1. SPLIT
        train_data = df.iloc[start_index : start_index + train_size]
        test_data = df.iloc[start_index + train_size : start_index + train_size + test_size]

        if len(test_data) == 0: break

        # 2. NORMALIZE
        scaler = StandardScaler()
        X_train = scaler.fit_transform(train_data[feature_cols])
        X_test = scaler.transform(test_data[feature_cols])

        y_train = train_data['target']
        y_test = test_data['target']

        # 3. TRAIN
        model.fit(X_train, y_train)

        # 4. PREDICT & MEASURE (TRAIN SET - The "Homework")
        train_preds = model.predict(X_train)
        train_acc = accuracy_score(y_train, train_preds)

        # Calculate Train PnL (How well it traded the history)
        train_ret = train_preds * train_data['ret_1d'].values
        pnl_train = (1 + train_ret).cumprod()[-1] - 1

        # 5. PREDICT & MEASURE (TEST SET - The "Final Exam")
        test_preds = model.predict(X_test)
        test_acc = accuracy_score(y_test, test_preds)

        # Calculate Test PnL (Real performance)
        test_ret = test_preds * test_data['ret_1d'].values
        pnl_test = (1 + test_ret).cumprod()[-1] - 1

        # 6. CALCULATE GAP
        acc_gap = train_acc - test_acc

        regime = detect_regime(test_data)

        results.append({
            'Period_Start': test_data.index[0],
            'Period_End': test_data.index[-1],
            'Regime': regime,
            'Train_Acc': train_acc,
            'Test_Acc': test_acc,
            'Gap_Acc': acc_gap,  # Important: High Gap = Overfitting
            'Train_PnL': pnl_train,
            'Test_PnL': pnl_test
        })

        start_index += test_size

    return pd.DataFrame(results)

# ==============================================================================
# 6. MAIN EXECUTION
# ==============================================================================
if __name__ == "__main__":

    # 1. Data
    print("📥 Downloading Data...")
    raw_df = yf.download('AAPL', start='2010-01-01', end='2024-01-01', progress=False, auto_adjust=False)
    if isinstance(raw_df.columns, pd.MultiIndex): raw_df.columns = raw_df.columns.droplevel(1)
    raw_df.columns = [c.lower() for c in raw_df.columns]
    raw_df['adjclose'] = raw_df['adj close'] if 'adj close' in raw_df.columns else raw_df['close']

    # 2. Features
    df = prepare_features(raw_df)
    features = ['ma_spread', 'mh_diff', 'h16', 'h32', 'vol_20', 'ret_1d', 'rsi']

    # 3. Filter
    final_features = remove_correlated_features(df, features, threshold=0.7)

    # 4. Run Backtest
    results_df = run_rolling_classification(df, final_features)

    # ==========================================================================
    # DISPLAY RESULTS
    # ==========================================================================

    # Formatting for cleaner display
    display_cols = ['Period_Start', 'Period_End', 'Regime', 'Train_Acc', 'Test_Acc', 'Gap_Acc', 'Train_PnL', 'Test_PnL']

    print("\n📉 PERIOD-BY-PERIOD RESULTS (Train vs Test)")
    print("="*120)
    # Using formatters to make percentages look nice
    pd.options.display.float_format = '{:,.2%}'.format
    print(results_df[display_cols].head(5).to_string(index=False))
    print("\n...\n")
    print(results_df[display_cols].tail(5).to_string(index=False))

    print("\n📊 ML RESULTS SUMMARY")
    print(f"Average PnL per period:   {results_df['Test_PnL'].mean():.2%}")
    print(f"Average Model Accuracy:   {results_df['Test_Acc'].mean():.2%}")

    # Calculate Win Rate
    win_rate = (results_df['Test_PnL'] > 0).sum() / len(results_df)
    print(f"Win Rate (Profitable Periods): {win_rate:.0%}")

    print("\n📊 OVERFITTING SUMMARY")
    print(f"Average Train Acc: {results_df['Train_Acc'].mean():.2%}")
    print(f"Average Test Acc:  {results_df['Test_Acc'].mean():.2%}")
    print(f"Average Gap:       {results_df['Gap_Acc'].mean():.2%}")

    if results_df['Gap_Acc'].mean() > 0.10:
        print("⚠️ WARNING: High Overfitting (>10% Gap). Consider reducing max_depth or features.")
    else:
        print("✅ Model looks stable (Gap < 10%).")

📥 Downloading Data...
🛠 Generating Technical Features...

🔍 CHECKING FEATURE CORRELATION...

🚀 STARTING ROLLING BACKTEST (Checking Overfitting)

📉 PERIOD-BY-PERIOD RESULTS (Train vs Test)
Period_Start Period_End                  Regime  Train_Acc  Test_Acc  Gap_Acc  Train_PnL  Test_PnL
  2012-08-06 2013-02-06 High Volatility (Panic)     64.88%    46.83%   18.06%    205.95%   -25.43%
  2013-02-07 2013-08-07 Bear Market (Calm Down)     68.85%    50.00%   18.85%     59.17%   -15.77%
  2013-08-08 2014-02-06   Bull Market (Calm Up)     68.25%    52.38%   15.87%     29.82%    -4.91%
  2014-02-07 2014-08-07   Bull Market (Calm Up)     67.06%    50.00%   17.06%    -29.29%    -7.39%
  2014-08-08 2015-02-06   Bull Market (Calm Up)     68.25%    46.03%   22.22%    -75.82%   -29.87%

...

Period_Start Period_End                  Regime  Train_Acc  Test_Acc  Gap_Acc  Train_PnL  Test_PnL
  2021-02-10 2021-08-10   Bull Market (Calm Up)     63.49%    50.79%   12.70%    -19.85%     7.88%
  2021-08-11 2

# 4. ML Trading Strategy: Regression with Rolling Cross-Validation (RAW)

In [14]:
# -*- coding: utf-8 -*-
"""
Full ML Framework: Rolling CV, Model Selection, RMSE, PnL, & Regimes
With Overfitting Analysis & Win Rate per Model
"""

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# ==============================================================================
# STEP 1: DATA ACQUISITION
# ==============================================================================
def get_data(symbol='AAPL'):
    print(f"📥 Downloading data for {symbol}...")
    df = yf.download(symbol, start='2010-01-01', end='2023-01-01', progress=False, auto_adjust=False)

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)

    # Standardize column names
    df.columns = [c.lower() for c in df.columns]

    # Use Adjusted Close if available, else Close
    col = 'adj close' if 'adj close' in df.columns else 'close'
    df['price'] = df[col]
    return df[['price']]

# ==============================================================================
# STEP 2: FEATURE ENGINEERING
# ==============================================================================
def prepare_features(df):
    data = df.copy()

    # --- Features (X) ---
    # 1. Momentum
    data['ret_1d'] = data['price'].pct_change()
    data['ret_5d'] = data['price'].pct_change(5)

    # 2. Volatility
    data['vol_20'] = data['ret_1d'].rolling(20).std()

    # 3. Trend (MA Ratio)
    data['ma_50_ratio'] = (data['price'] / data['price'].rolling(50).mean()) - 1

    # --- Target (Y) ---
    # Predict EXACT return of tomorrow
    data['target'] = data['ret_1d'].shift(-1)

    data = data.dropna()
    return data

# ==============================================================================
# STEP 3: REGIME DETECTION
# ==============================================================================
def detect_regime(data_slice):
    vol = data_slice['ret_1d'].std() * np.sqrt(252)

    if vol > 0.30:
        return "High Volatility (Panic)"
    elif data_slice['price'].iloc[-1] > data_slice['price'].iloc[0]:
        return "Bull Market (Calm Up)"
    else:
        return "Bear Market (Calm Down)"

# ==============================================================================
# STEP 4: ROLLING CV & MODEL SELECTION
# ==============================================================================
def run_full_analysis(df, models_to_test):

    # Define features
    features = ['ret_1d', 'ret_5d', 'vol_20', 'ma_50_ratio']

    # Rolling Window Settings
    train_months = 24
    test_months = 6
    train_size = train_months * 21
    test_size = test_months * 21

    all_results = []

    # --- LOOP: Rolling Time Windows ---
    start_index = 0
    while start_index + train_size + test_size < len(df):

        # 1. SPLIT TIME
        train_data = df.iloc[start_index : start_index + train_size]
        test_data = df.iloc[start_index + train_size : start_index + train_size + test_size]

        if len(test_data) == 0: break

        # 2. CONTEXT
        current_regime = detect_regime(test_data)

        # 3. LOOP MODELS
        for model_name, model_obj in models_to_test.items():

            # A. TRAIN
            X_train = train_data[features]
            y_train = train_data['target']
            model_obj.fit(X_train, y_train)

            # Metrics (In-Sample)
            pred_train = model_obj.predict(X_train)
            rmse_train = np.sqrt(mean_squared_error(y_train, pred_train))

            sig_train = np.where(pred_train > 0, 1, 0)
            ret_train = sig_train * y_train
            pnl_train = (1 + ret_train).cumprod().iloc[-1] - 1

            # B. TEST
            X_test = test_data[features]
            y_test = test_data['target']

            pred_test = model_obj.predict(X_test)

            # Metrics (Out-of-Sample)
            rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))

            sig_test = np.where(pred_test > 0, 1, 0)
            ret_test = sig_test * y_test
            pnl_test = (1 + ret_test).cumprod().iloc[-1] - 1

            # Gap
            rmse_gap = rmse_test - rmse_train

            # Store
            all_results.append({
                'Period_Start': test_data.index[0],
                'Period_End': test_data.index[-1],
                'Model': model_name,
                'Regime': current_regime,
                'Train_RMSE': rmse_train,
                'Test_RMSE': rmse_test,
                'RMSE_Gap': rmse_gap,
                'Train_PnL': pnl_train,
                'Test_PnL': pnl_test
            })

        # Slide Window
        start_index += test_size

    # Add Year
    results_df = pd.DataFrame(all_results)
    results_df['Year'] = results_df['Period_End'].dt.year

    return results_df

# ==============================================================================
# STEP 5: MAIN EXECUTION
# ==============================================================================
if __name__ == "__main__":

    # 1. Prepare Data
    raw_df = get_data('AAPL')
    df = prepare_features(raw_df)

    # 2. Define Models
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(n_estimators=50, max_depth=3, random_state=42)
    }

    print("\n🚀 STARTING ROLLING BACKTEST...")
    results_df = run_full_analysis(df, models)

    # ==========================================================================
    # STEP 6: DISPLAY RESULTS
    # ==========================================================================

    # Formatting
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    pd.options.display.float_format = '{:.6f}'.format

    # 1. Detailed Table
    print("\n📉 PERIOD-BY-PERIOD RESULTS (Sample)")
    print("="*120)
    cols = ['Period_Start', 'Period_End', 'Model', 'Regime', 'Train_RMSE', 'Test_RMSE', 'RMSE_Gap', 'Train_PnL', 'Test_PnL', 'Year']
    print(results_df[cols].head(10).to_string(index=False))
    print("\n...\n")
    print(results_df[cols].tail(6).to_string(index=False))

    # 2. Model Stats Summary
    print("\n📊 REGRESSION SUMMARY (Average)")
    summary = results_df.groupby('Model')[['Train_RMSE', 'Test_RMSE', 'RMSE_Gap', 'Train_PnL', 'Test_PnL']].mean()
    print(summary)

    # 3. WIN RATE ANALYSIS (PER MODEL) - THE NEW PART
    print("\n🎯 WIN RATE ANALYSIS (Per Model)")
    print("="*60)

    for model in results_df['Model'].unique():
        # Filter for the specific model
        model_data = results_df[results_df['Model'] == model]

        # Calculate Wins (Where Test PnL > 0)
        total_periods = len(model_data)
        winning_periods = (model_data['Test_PnL'] > 0).sum()

        # Calculate Percentage
        if total_periods > 0:
            win_rate = winning_periods / total_periods
        else:
            win_rate = 0.0

        print(f"   {model:<20}: {win_rate:.1%} ({winning_periods}/{total_periods} periods profitable)")

📥 Downloading data for AAPL...

🚀 STARTING ROLLING BACKTEST...

📉 PERIOD-BY-PERIOD RESULTS (Sample)
Period_Start Period_End             Model                  Regime  Train_RMSE  Test_RMSE  RMSE_Gap  Train_PnL  Test_PnL  Year
  2012-03-14 2012-09-11 Linear Regression   Bull Market (Calm Up)    0.016400   0.017639  0.001239   1.827783  0.144550  2012
  2012-03-14 2012-09-11     Random Forest   Bull Market (Calm Up)    0.015067   0.017355  0.002288   4.662402  0.184748  2012
  2012-09-12 2013-03-15 Linear Regression High Volatility (Panic)    0.016177   0.023174  0.006997   1.626431 -0.288746  2013
  2012-09-12 2013-03-15     Random Forest High Volatility (Panic)    0.014310   0.026255  0.011945   3.579226 -0.052388  2013
  2013-03-18 2013-09-13 Linear Regression   Bull Market (Calm Up)    0.018775   0.016835 -0.001940   0.478269 -0.126387  2013
  2013-03-18 2013-09-13     Random Forest   Bull Market (Calm Up)    0.016897   0.017861  0.000964   3.363598 -0.070813  2013
  2013-09-16 2014-

# 5. ML Trading Strategy: Regressions with Rolling Cross-Validation (New Features + Normalized)

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# ==============================================================================
# 1. HELPER & 2. FEATURES (Same as above)
# ==============================================================================
def get_hurst_exponent(time_series):
    lags = range(2, 20)
    tau = []
    for lag in lags:
        if len(time_series) > lag:
            diff = np.subtract(time_series[lag:], time_series[:-lag])
            tau.append(np.sqrt(np.std(diff)))
        else:
            return 0.5
    try:
        m = np.polyfit(np.log(lags), np.log(tau), 1)
        return m[0] * 2.0
    except:
        return 0.5

def prepare_features(df):
    print("🛠 Generating Technical Features...")
    data = df.copy()
    data['ma_15'] = data['adjclose'].rolling(window=15).mean()
    data['ma_150'] = data['adjclose'].rolling(window=150).mean()
    data['ma_spread'] = (data['ma_15'] - data['ma_150']) / data['ma_150']

    log_ret = np.log(data['adjclose'] / data['adjclose'].shift(1))
    data['h16'] = log_ret.rolling(window=16).apply(get_hurst_exponent, raw=True)
    data['h32'] = log_ret.rolling(window=32).apply(get_hurst_exponent, raw=True)
    data['mh_diff'] = data['h32'] - data['h16']

    delta = data['adjclose'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    data['rsi'] = 100 - (100 / (1 + rs))

    data['ret_1d'] = data['adjclose'].pct_change()
    data['vol_20'] = data['ret_1d'].rolling(20).std()

    # TARGET (Exact Return)
    data['target'] = data['ret_1d'].shift(-1)

    data = data.dropna()
    return data

def remove_correlated_features(df, feature_cols, threshold=0.7):
    print("\n🔍 CHECKING FEATURE CORRELATION...")
    df_feat = df[feature_cols]
    corr_matrix = df_feat.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return [f for f in feature_cols if f not in to_drop]

def detect_regime(data_slice):
    vol = data_slice['ret_1d'].std() * np.sqrt(252)
    if vol > 0.30:
        return "High Volatility (Panic)"
    elif data_slice['adjclose'].iloc[-1] > data_slice['adjclose'].iloc[0]:
        return "Bull Market (Calm Up)"
    else:
        return "Bear Market (Calm Down)"

# ==============================================================================
# 5. ROLLING REGRESSION (Multi-Model with Train/Test)
# ==============================================================================
def run_rolling_regression(df, feature_cols):

    print(f"\n🚀 STARTING ROLLING REGRESSION (Checking Overfitting)")
    print("="*80)

    train_months = 24
    test_months = 6
    train_size = train_months * 21
    test_size = test_months * 21

    results = []

    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=3, random_state=42)
    }

    start_index = 0
    while start_index + train_size + test_size < len(df):

        train_data = df.iloc[start_index : start_index + train_size]
        test_data = df.iloc[start_index + train_size : start_index + train_size + test_size]

        if len(test_data) == 0: break

        # Normalize
        scaler = StandardScaler()
        X_train = scaler.fit_transform(train_data[feature_cols])
        X_test = scaler.transform(test_data[feature_cols])
        y_train = train_data['target']
        y_test = test_data['target']

        regime = detect_regime(test_data)

        for model_name, model_obj in models.items():
            model_obj.fit(X_train, y_train)

            # --- TRAIN PERFORMANCE ---
            pred_train = model_obj.predict(X_train)
            rmse_train = np.sqrt(mean_squared_error(y_train, pred_train))

            sig_train = np.where(pred_train > 0, 1, 0)
            ret_train = sig_train * y_train
            pnl_train = (1 + ret_train).cumprod().iloc[-1] - 1

            # --- TEST PERFORMANCE ---
            pred_test = model_obj.predict(X_test)
            rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))

            sig_test = np.where(pred_test > 0, 1, 0)
            ret_test = sig_test * y_test
            pnl_test = (1 + ret_test).cumprod().iloc[-1] - 1

            # --- GAP ---
            # For RMSE, we want Train and Test to be close.
            # If Train is 0.01 and Test is 0.05, that's overfitting.
            rmse_gap = rmse_test - rmse_train

            results.append({
                'Period_Start': test_data.index[0],
                'Period_End': test_data.index[-1],
                'Model': model_name,
                'Regime': regime,
                'Train_RMSE': rmse_train,
                'Test_RMSE': rmse_test,
                'RMSE_Gap': rmse_gap, # Positive is normal, but shouldn't be too big
                'Train_PnL': pnl_train,
                'Test_PnL': pnl_test
            })

        start_index += test_size

    results_df = pd.DataFrame(results)
    results_df['Year'] = results_df['Period_End'].dt.year

    cols = ['Period_Start', 'Period_End', 'Model', 'Regime', 'Train_RMSE', 'Test_RMSE', 'RMSE_Gap', 'Train_PnL', 'Test_PnL', 'Year']
    return results_df[cols]

# ==============================================================================
# 6. MAIN EXECUTION
# ==============================================================================
if __name__ == "__main__":

    print("📥 Downloading Data...")
    raw_df = yf.download('AAPL', start='2010-01-01', end='2024-01-01', progress=False, auto_adjust=False)
    if isinstance(raw_df.columns, pd.MultiIndex): raw_df.columns = raw_df.columns.droplevel(1)
    raw_df.columns = [c.lower() for c in raw_df.columns]
    raw_df['adjclose'] = raw_df['adj close'] if 'adj close' in raw_df.columns else raw_df['close']

    df = prepare_features(raw_df)
    features = ['ma_spread', 'mh_diff', 'h16', 'h32', 'vol_20', 'ret_1d', 'rsi']
    final_features = remove_correlated_features(df, features, threshold=0.7)

    results_df = run_rolling_regression(df, final_features)

    # ==========================================================================
    # DISPLAY RESULTS
    # ==========================================================================

    print("\n📉 PERIOD-BY-PERIOD RESULTS (Sample)")
    print("="*120)
    # pd.options.display.float_format = '{:.5f}'.format # Precise for RMSE
    print(results_df.head(5).to_string(index=False))
    print("\n📊 ML RESULTS SUMMARY (REGRESSION)")
    print(f"Average PnL per period:   {results_df['Test_PnL'].mean():.2%}")
    print(f"Average RMSE (Error):     {results_df['Test_RMSE'].mean():.5f}")
    # Baseline RMSE is not calculated or stored in this version of results_df, so commenting it out
    # print(f"Baseline RMSE (Zero):     {results_df['Baseline_RMSE'].mean():.5f}")

    print("\n📊 REGRESSION SUMMARY (Average)")
    summary = results_df.groupby('Model')[['Train_RMSE', 'Test_RMSE', 'RMSE_Gap', 'Train_PnL', 'Test_PnL']].mean()
    print(summary)

     # 3. WIN RATE ANALYSIS (PER MODEL) - THE NEW PART
    print("\n🎯 WIN RATE ANALYSIS (Per Model)")
    print("="*60)

    for model in results_df['Model'].unique():
        # Filter for the specific model
        model_data = results_df[results_df['Model'] == model]

        # Calculate Wins (Where Test PnL > 0)
        total_periods = len(model_data)
        winning_periods = (model_data['Test_PnL'] > 0).sum()

        # Calculate Percentage
        if total_periods > 0:
            win_rate = winning_periods / total_periods
        else:
            win_rate = 0.0

        print(f"   {model:<20}: {win_rate:.1%} ({winning_periods}/{total_periods} periods profitable)")

📥 Downloading Data...
🛠 Generating Technical Features...

🔍 CHECKING FEATURE CORRELATION...

🚀 STARTING ROLLING REGRESSION (Checking Overfitting)

📉 PERIOD-BY-PERIOD RESULTS (Sample)
Period_Start Period_End             Model                  Regime  Train_RMSE  Test_RMSE  RMSE_Gap  Train_PnL  Test_PnL  Year
  2012-08-06 2013-02-06 Linear Regression High Volatility (Panic)    0.016216   0.022880  0.006664   1.776568 -0.262639  2013
  2012-08-06 2013-02-06     Random Forest High Volatility (Panic)    0.014967   0.023002  0.008035   4.159717 -0.187683  2013
  2013-02-07 2013-08-07 Linear Regression Bear Market (Calm Down)    0.018678   0.015522 -0.003156   0.674338  0.074437  2013
  2013-02-07 2013-08-07     Random Forest Bear Market (Calm Down)    0.017194   0.016731 -0.000463   3.686489 -0.076754  2013
  2013-08-08 2014-02-06 Linear Regression   Bull Market (Calm Up)    0.018910   0.016466 -0.002444   0.790473  0.150920  2014

📊 ML RESULTS SUMMARY (REGRESSION)
Average PnL per period:   